In [7]:
import numpy as np
import random
from evoman.environment import Environment
from demo_controller import player_controller
import os


In [8]:

n_hidden_neurons=10
#n_weights = (env.get_num_sensors() + 1) * n_hidden_neurons + (n_hidden_neurons + 1) * 5
n_weights=265
experiment_name = 'generalist1'
if not os.path.exists(experiment_name):
    os.makedirs(experiment_name)

In [24]:
def initialize_population(population_size, n_vars):
    population=[]
    individual_mutation_rates=[]
    for i in range(population_size):
        population.append(np.random.uniform(-1, 1, n_vars)) 
    return np.array(population)

pop=initialize_population(3,265) # for debugging

def simulation(ind,enemies=[2,4,6]): # just for debugging purpose set enemies as 2, 4, 6
    ''' simulates the environment and evaluates the fitness score against group of enemies
    returns fitness_score: list of fitness scores against each enemy and also average fitness too
    basically fitness_score[0]= fitness score upon enemies[0] and so on 
    and fitness_score[4] = mean(fitness_score) - standard deviation(fitness_score)'''

    individual_scores=[] 
    for i in range(len(enemies)):
      env = Environment(experiment_name=experiment_name,
                  enemies=[enemies[i]],
                  playermode="ai",
                  player_controller=player_controller(n_hidden_neurons),
                  enemymode="static",
                  level=2,
                  speed="fastest",
                  visuals=False)
      f,p,e,t = env.play(pcont=ind)
      individual_scores.append(f)
    
    avg_fitness = env.cons_multi(np.array(individual_scores))
    individual_scores.append(avg_fitness)

    return individual_scores


def evaluation(population, enemies:list):
   '''calls the simulation function for each individual in population 
      returns np array of fitness scores for whole population'''
   
   fitness_scores=[]
   for ind in population:
      individual_scores = simulation(ind,enemies)
      fitness_scores.append(individual_scores)

   return np.array(fitness_scores)
    
evaluation(pop,[2,4,6]) # debugging




      


MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.


array([[-5.83481074, -7.24779258, -6.18620862, -7.02358129],
       [-5.79605775, -5.0689042 , -4.77912349, -5.64246461],
       [91.49061166, 21.40528862, 21.9062498 , 12.01295432]])